In [1]:
import pandas as pd
import numpy as np

#To set up similarity matrix
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

#Could perform train_test_split and metrics.accuracy_score test if needed.   
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC 
from sklearn import metrics

In [6]:
df = pd.read_csv("../data_cleaning/export/movie_db.csv")
df.head()

,index,title,id,release_date,cast,percentage_top_5_female_led,total_top_5_female_led,total_female_actors,total_actors,percentage_female_cast,...,percentage_us,international,original_language,languages,total_genres,genres,runtime,revenue,budget,budget_bins
0,0,The Suicide Squad,436969,7/28/21,Margot Robbie Idris Elba John Cena Joel Kinnam...,0.2,1,11,44,0.250000,...,1.0,0,en,English Spanish,4,Action Adventure Fantasy Comedy,132,118084747,180000000,151m to 380m
1,1,Jungle Cruise,451048,7/28/21,Dwayne Johnson Emily Blunt Edgar Ramírez Jack ...,0.2,1,11,67,0.164179,...,1.0,0,en,English Spanish Portuguese,4,Adventure Fantasy Comedy Action,127,154342554,200000000,151m to 380m
2,2,Black Widow,497698,7/7/21,Scarlett Johansson Florence Pugh Rachel Weisz ...,0.6,3,33,96,0.343750,...,1.0,0,en,English Russian,4,Action Adventure Thriller Science Fiction,134,367799531,200000000,151m to 380m
3,3,Space Jam: A New Legacy,379686,7/8/21,LeBron James Don Cheadle Cedric Joe Jeff Bergm...,0.0,0,9,37,0.243243,...,1.0,0,en,English,4,Animation Comedy Family Science Fiction,115,144882738,161900000,151m to 380m
4,4,The Boss Baby: Family Business,459151,7/1/21,Alec Baldwin James Marsden Amy Sedaris Jeff Go...,0.4,2,4,25,0.160000,...,1.0,0,en,English,4,Animation Comedy Adventure Family,107,70324240,82000000,51m to 150m


In [7]:
features = df[['index','title','release_date','cast','total_top_5_female_led','total_female_actors','percentage_female_cast','international','original_language','languages','genres','budget','budget_bins','popularity','tagline','keywords','production_companies','production_company_origin_country']]

features.head()

,index,title,release_date,cast,total_top_5_female_led,total_female_actors,percentage_female_cast,international,original_language,languages,genres,budget,budget_bins,popularity,tagline,keywords,production_companies,production_company_origin_country
0,0,The Suicide Squad,7/28/21,Margot Robbie Idris Elba John Cena Joel Kinnam...,1,11,0.250000,0,en,English Spanish,Action Adventure Fantasy Comedy,180000000,151m to 380m,7243.123,They're dying to save the world.,dc comics monster anti hero secret mission sup...,DC Entertainment DC Films Atlas Entertainment ...,"['US', 'US', 'US', 'US', 'US', 'US']"
1,1,Jungle Cruise,7/28/21,Dwayne Johnson Emily Blunt Edgar Ramírez Jack ...,1,11,0.164179,0,en,English Spanish Portuguese,Adventure Fantasy Comedy Action,200000000,151m to 380m,6680.739,NaN,jungle riverboat early 1900s based on theme pa...,Walt Disney Pictures Seven Bucks Productions D...,"['US', 'US', 'US', 'US', 'US', 'US']"
2,2,Black Widow,7/7/21,Scarlett Johansson Florence Pugh Rachel Weisz ...,3,33,0.343750,0,en,English Russian,Action Adventure Thriller Science Fiction,200000000,151m to 380m,2564.501,Her world. Her secrets. Her legacy.,spy assassin hero russia kgb based on comic fe...,Marvel Studios,['US']
3,3,Space Jam: A New Legacy,7/8/21,LeBron James Don Cheadle Cedric Joe Jeff Bergm...,0,9,0.243243,0,en,English,Animation Comedy Family Science Fiction,161900000,151m to 380m,2327.176,Welcome to the Jam!,sports basketball cartoon game live action and...,Warner Animation Group SpringHill Entertainmen...,"['US', 'US', 'US', 'US']"
4,4,The Boss Baby: Family Business,7/1/21,Alec Baldwin James Marsden Amy Sedaris Jeff Go...,2,4,0.160000,0,en,English,Animation Comedy Adventure Family,82000000,51m to 150m,1759.371,Playtime is over.,baby sequel family,DreamWorks Animation Universal Pictures,"['US', 'US']"


## Create combined_features row to run similarity matrix

In [8]:
def combine_features(row):
    return row['cast']+" "+row['keywords']+" "+row['genres']+" "+row['tagline']+" "+row['production_companies']+" "+row['production_company_origin_country']

In [9]:
for feature in features:
    features = features.fillna('')
    features['combined_features'] = features.apply(combine_features, axis=1)

features.head(50)

,index,title,release_date,cast,total_top_5_female_led,total_female_actors,percentage_female_cast,international,original_language,languages,genres,budget,budget_bins,popularity,tagline,keywords,production_companies,production_company_origin_country,combined_features
0,0,The Suicide Squad,7/28/21,Margot Robbie Idris Elba John Cena Joel Kinnam...,1,11,0.250000,0,en,English Spanish,Action Adventure Fantasy Comedy,180000000,151m to 380m,7243.123,They're dying to save the world.,dc comics monster anti hero secret mission sup...,DC Entertainment DC Films Atlas Entertainment ...,"['US', 'US', 'US', 'US', 'US', 'US']",Margot Robbie Idris Elba John Cena Joel Kinnam...
1,1,Jungle Cruise,7/28/21,Dwayne Johnson Emily Blunt Edgar Ramírez Jack ...,1,11,0.164179,0,en,English Spanish Portuguese,Adventure Fantasy Comedy Action,200000000,151m to 380m,6680.739,,jungle riverboat early 1900s based on theme pa...,Walt Disney Pictures Seven Bucks Productions D...,"['US', 'US', 'US', 'US', 'US', 'US']",Dwayne Johnson Emily Blunt Edgar Ramírez Jack ...
2,2,Black Widow,7/7/21,Scarlett Johansson Florence Pugh Rachel Weisz ...,3,33,0.343750,0,en,English Russian,Action Adventure Thriller Science Fiction,200000000,151m to 380m,2564.501,Her world. Her secrets. Her legacy.,spy assassin hero russia kgb based on comic fe...,Marvel Studios,['US'],Scarlett Johansson Florence Pugh Rachel Weisz ...
3,3,Space Jam: A New Legacy,7/8/21,LeBron James Don Cheadle Cedric Joe Jeff Bergm...,0,9,0.243243,0,en,English,Animation Comedy Family Science Fiction,161900000,151m to 380m,2327.176,Welcome to the Jam!,sports basketball cartoon game live action and...,Warner Animation Group SpringHill Entertainmen...,"['US', 'US', 'US', 'US']",LeBron James Don Cheadle Cedric Joe Jeff Bergm...
4,4,The Boss Baby: Family Business,7/1/21,Alec Baldwin James Marsden Amy Sedaris Jeff Go...,2,4,0.160000,0,en,English,Animation Comedy Adventure Family,82000000,51m to 150m,1759.371,Playtime is over.,baby sequel family,DreamWorks Animation Universal Pictures,"['US', 'US']",Alec Baldwin James Marsden Amy Sedaris Jeff Go...
5,5,The Forever Purge,6/30/21,Josh Lucas Ana de la Reguera Tenoch Huerta Wil...,2,7,0.175000,0,en,English Spanish,Horror Action Thriller,18000000,16m to 50m,1379.329,The rules are broken.,sequel prequel murder purge,Blumhouse Productions Platinum Dunes Universal...,"['US', 'US', 'US', 'CN', 'US']",Josh Lucas Ana de la Reguera Tenoch Huerta Wil...
6,6,Don't Breathe 2,8/12/21,Stephen Lang Brendan Sexton III Madelyn Grace ...,1,1,0.090909,0,en,English Hungarian,Thriller Horror,10000000,0 to 15m,1312.696,Bad things happen to bad people.,home invasion sin sequel gunfight blind home i...,Ghost House Pictures Stage 6 Films Good Univer...,"['US', 'US', 'US', 'US', 'US']",Stephen Lang Brendan Sexton III Madelyn Grace ...
7,7,Cruella,5/26/21,Emma Stone Emma Thompson Joel Fry Paul Walter ...,2,9,0.300000,0,en,English,Comedy Crime,200000000,151m to 380m,1125.416,Hello Cruel World,1970s anti hero villain punk rock fashion desi...,Walt Disney Pictures,['US'],Emma Stone Emma Thompson Joel Fry Paul Walter ...
8,8,A Quiet Place Part II,5/21/21,Emily Blunt Cillian Murphy Millicent Simmonds ...,2,7,0.280000,0,en,English,Science Fiction Thriller Horror,61000000,51m to 150m,825.128,Silence is not enough.,island radio transmission post-apocalyptic fut...,Platinum Dunes Sunday Night Paramount,"['US', 'US', 'US']",Emily Blunt Cillian Murphy Millicent Simmonds ...
9,9,The Conjuring: The Devil Made Me Do It,5/25/21,Patrick Wilson Vera Farmiga Ruairi O'Connor Sa...,2,13,0.406250,0,en,English Portuguese,Horror Mystery Thriller,39000000,16m to 50m,809.518,The demonic case that shocked America.,horror religious horror,New Line Cinema The Safran Company Atomic Mons...,"['US', 'US', 'US', 'US']",Patrick Wilson Vera Farmiga Ruairi O'Connor Sa...


## Create CountVectorizer matrix and Similarity matrix

In [10]:
#Create new CountVectorizer matrix
cv = CountVectorizer()
count_matrix = cv.fit_transform(features['combined_features'])

In [11]:
#Obtain cosine similarity matrix from the count matrix
cosine_sim = cosine_similarity(count_matrix)

In [12]:
# get movie title from movie index and vice-versa
def get_title_from_index(index):
    return features[features.index == index]["title"].values[0]
def get_index_from_title(title):
    return features[features.title == title]["index"].values[0]

## Find Similarity Score
- First get title of movie that user likes and find the index of that movie. 
- Then access row corresponding to this movie in the similarity matrix. 
- Thus, we will get the similarity scores of all other movies from the current movie.
- Then we will enumerate through all similarity scores of that movie to make a tuple of movie index and similarity scores.
- This will convert row of similarity scores from [1 0.5 0.2 0.9] to this- [(0, 1) (1, 0.5) (2, 0.2) (3, 0.9)] . Here, each item is in this form: (movie index, similarity score).

In [13]:
#assesses row corresponding to given movie to find all similarity scores for that movie and then enmerates over it.
movie_user_likes = "Toy Story 3"
movie_index = get_index_from_title(movie_user_likes)
similar_movies = list(enumerate(cosine_sim[movie_index])) 
similar_movies

[(0, 0.0901127113779166),
 (1, 0.08365153132187089),
 (2, 0.04643634602125721),
 (3, 0.11378511742338251),
 (4, 0.08504865111251246),
 (5, 0.06905413026317789),
 (6, 0.061402364516266905),
 (7, 0.07838618016696204),
 (8, 0.07456437192014401),
 (9, 0.048304589153964794),
 (10, 0.03327791628198609),
 (11, 0.042294442611014484),
 (12, 0.0690541302631779),
 (13, 0.06237449404683604),
 (14, 0.08291849892341731),
 (15, 0.0),
 (16, 0.08692868576629595),
 (17, 0.06388765649999399),
 (18, 0.09224150860310645),
 (19, 0.06891687876063168),
 (20, 0.10746372201097237),
 (21, 0.10044944046678454),
 (22, 0.09304842103984706),
 (23, 0.10359079584873414),
 (24, 0.02339043526756351),
 (25, 0.09065483194721177),
 (26, 0.040246708254332256),
 (27, 0.15452362609131381),
 (28, 0.03423683940087303),
 (29, 0.13120407626405034),
 (30, 0.06639400022069858),
 (31, 0.04623207306786785),
 (32, 0.07787611974767476),
 (33, 0.05),
 (34, 0.0861460984507896),
 (35, 0.05385770740428468),
 (36, 0.06639400022069858),
 (37

In [14]:
#Sort the list similar_movies accroding to similarity scores in descending order. Since the most similar movie to a given movie is itself, discard the first elements after sorting movies.
sorted_similar_movies = sorted(similar_movies, key=lambda
                              x:x[1], reverse=True)[1:]
sorted_similar_movies

[(131, 0.5065247452736259),
 (93, 0.3942103318208259),
 (89, 0.3869412129822142),
 (230, 0.33449032458310296),
 (410, 0.33037086020703704),
 (211, 0.3266958198847344),
 (391, 0.30180083472774627),
 (69, 0.2857142857142858),
 (237, 0.26562660124543),
 (803, 0.2529121637430944),
 (213, 0.244056730175554),
 (180, 0.22736665130467296),
 (977, 0.22544859207627124),
 (1678, 0.2194302211769791),
 (227, 0.21459876881973805),
 (378, 0.21303452115065533),
 (303, 0.21044171232366055),
 (549, 0.2079413259807166),
 (1787, 0.20351933162035313),
 (3120, 0.20123572383619492),
 (458, 0.19823785111334077),
 (202, 0.19200189632438996),
 (289, 0.19150116029469202),
 (2192, 0.18754070335443593),
 (233, 0.18723608468379271),
 (866, 0.18420709354880074),
 (2273, 0.1834729889175452),
 (901, 0.1828754635743658),
 (2736, 0.1826960238104674),
 (2562, 0.1820877694741164),
 (1011, 0.18050042628565835),
 (3360, 0.17959152544456344),
 (2673, 0.17584153490791937),
 (149, 0.17530461293723326),
 (763, 0.175084175995506

In [15]:
#run a loop to print first 50 entries from sorted_similar_movies list
i=0
print("Top 50 Similar movies to "+movie_user_likes+" are:\n")
for movie in sorted_similar_movies:
    print(get_title_from_index(movie[0]))
    i=i+1
    if i>49:
        break

Top 50 Similar movies to Toy Story 3 are:

Toy Story 2
Toy Story 4
Toy Story
Monsters University
Cars
Up
WALL·E
Monsters, Inc.
A Bug's Life
Minions
Inside Out
The Incredibles
Horton Hears a Who!
Zodiac
Cars 2
Despicable Me 3
Despicable Me 2
Cloudy with a Chance of Meatballs
Surf's Up
Around the World in Eighty Days
The Grinch
The Emperor's New Groove
Ice Age: The Meltdown
Problem Child
Transformers: The Last Knight
Independence Day
The Tigger Movie
Superman
Small Soldiers
Trading Places
The Smurfs
The Hours
JFK
101 Dalmatians
Looney Tunes: Back in Action
Die Hard 2
Public Enemies
Return of the Jedi
Pocahontas
Planes
Stargate
Wreck-It Ralph
Transformers: Dark of the Moon
RoboCop 2
The Flintstones in Viva Rock Vegas
Ted
Transformers: Revenge of the Fallen
Quest for Camelot
The Empire Strikes Back
*batteries not included
